In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import json
import ast
from ast import literal_eval
from IPython.display import display
import warnings
warnings.filterwarnings('ignore', category=UserWarning)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 1. 데이터 로드
print("TMDB 5000 Movies Dataset 데이터 전처리 시작...")
df = pd.read_csv('tmdb_5000_movies.csv')

print("tmdb_5000_credits 데이터 전처리 시작...")
df = pd.read_csv('tmdb_5000_credits.csv')


print("\n=== 기본 데이터 정보 ===")
print(f"데이터 형태: {df.shape}")
print(f"컬럼 수: {df.shape[1]}")
print(f"행 수: {df.shape[0]}")

# 2. 데이터 개요 확인
print("\n=== 컬럼 정보 ===")
print(df.info())

print("\n=== 기본 통계 ===")
print(df.describe())

print("\n=== 결측값 확인 ===")
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage': missing_percent
}).sort_values('Missing Count', ascending=False)
print(missing_df[missing_df['Missing Count'] > 0])

# 3. 데이터 전처리 함수들
def parse_json_column(df, column_name):
    """JSON 형태의 문자열을 파싱하여 리스트로 변환"""
    def safe_literal_eval(x):
        if pd.isna(x) or x == '':
            return []
        try:
            return ast.literal_eval(x)
        except:
            return []

    df[column_name + '_parsed'] = df[column_name].apply(safe_literal_eval)
    return df

def extract_names_from_json(df, column_name):
    """JSON 리스트에서 name 필드만 추출"""
    def extract_names(x):
        if isinstance(x, list) and len(x) > 0:
            return [item.get('name', '') for item in x if isinstance(item, dict)]
        return []

    df[column_name + '_names'] = df[column_name + '_parsed'].apply(extract_names)
    return df

# 4. 데이터 전처리 실행
print("\n=== 데이터 전처리 시작 ===")

# 날짜 컬럼 처리
if 'release_date' in df.columns:
    df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
print("✓ 날짜 컬럼 전처리 완료")

# JSON 컬럼들을 리스트로 변환 처리 (genres, keywords, production_companies, production_countries, spoken_languages)
json_columns = ['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages']

for col in json_columns:
    # 문자열 형태의 JSON 리스트를 실제 파이썬 리스트로 변환
    df[col] = df[col].apply(ast.literal_eval)
    # 각 리스트 항목의 'name' 필드만 추출하여 새 리스트로 변환
    df[col] = df[col].apply(lambda items: [item.get('name') for item in items])
print("✓ 컬럼들을 리스트로 변환 처리 완료")

# 수치형 데이터 처리
numeric_columns = ['budget', 'revenue', 'runtime', 'popularity', 'vote_average', 'vote_count']
for col in numeric_columns:
    if col in df.columns:
        # pd.to_numeric을 이용해 적절한 수치형 데이터로 자동 변환
        df[col] = pd.to_numeric(df[col], errors='coerce')
print("✓ 수치형 데이터 처리 완료")

# 파생 변수 생성
if 'budget' in df.columns and 'revenue' in df.columns:
    df['profit'] = df['revenue'] - df['budget']
    df['roi'] = df['profit'] / df['budget'].replace(0, np.nan)  # ROI (투자수익률)
    print("✓ 파생 변수 생성 완료 (profit, roi)")

# 칼럼 순서 변경
new_order = ['id', 'title', 'original_title', 'release_date', 'budget', 'revenue', 'profit', 'roi',
             'runtime', 'popularity', 'vote_average', 'vote_count', 'status',
             'genres', 'keywords', 'production_companies', 'production_countries', 
             'spoken_languages', 'original_language', 'overview', 'tagline', 'homepage']

df = df[new_order]

TMDB 5000 Movies Dataset 데이터 전처리 시작...

=== 기본 데이터 정보 ===
데이터 형태: (4803, 20)
컬럼 수: 20
행 수: 4803

=== 컬럼 정보 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime     

In [19]:
print("tmdb_5000_credits 데이터 전처리 시작...")
df = pd.read_csv('tmdb_5000_credits.csv')

print("\n=== 기본 데이터 정보 ===")
print(f"데이터 형태: {df.shape}")
print(f"컬럼 수: {df.shape[1]}")
print(f"행 수: {df.shape[0]}")

# 2. 데이터 개요 확인
print("\n=== 컬럼 정보 ===")
print(df.info())

print("\n=== 기본 통계 ===")
print(df.describe())

print("\n=== 결측값 확인 ===")
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage': missing_percent
}).sort_values('Missing Count', ascending=False)
print(missing_df[missing_df['Missing Count'] > 0])


tmdb_5000_credits 데이터 전처리 시작...

=== 기본 데이터 정보 ===
데이터 형태: (4803, 4)
컬럼 수: 4
행 수: 4803

=== 컬럼 정보 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  4803 non-null   int64 
 1   title     4803 non-null   object
 2   cast      4803 non-null   object
 3   crew      4803 non-null   object
dtypes: int64(1), object(3)
memory usage: 150.2+ KB
None

=== 기본 통계 ===
            movie_id
count    4803.000000
mean    57165.484281
std     88694.614033
min         5.000000
25%      9014.500000
50%     14629.000000
75%     58610.500000
max    459488.000000

=== 결측값 확인 ===
Empty DataFrame
Columns: [Missing Count, Missing Percentage]
Index: []


In [20]:
df['cast'] = df['cast'].apply(ast.literal_eval)
df['crew'] = df['crew'].apply(ast.literal_eval)

df.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de..."


In [ ]:
pprint(response.json())

# 과제 1: 장르별 영화 트렌드 및 흥행 성공 요인 분석
최근 20년간 영화 장르별 트렌드 변화를 분석하고, 각 장르에서 흥행에 성공한 영화들의 공통 요소(예산, 런타임, 출연진, 개봉 시기 등)를 파악하세요.

In [ ]:
df = ['id', 'title', 'original_title', 'release_date', 'budget', 'revenue', 'profit', 'roi',
             'runtime', 'popularity', 'vote_average', 'vote_count', 'status',
             'genres', 'keywords', 'production_companies', 'production_countries', 
             'spoken_languages', 'original_language', 'overview', 'tagline', 'homepage']

In [8]:
print("\n=== 컬럼 정보 ===")
print(df.info())

print("\n=== 기본 통계 ===")
print(df.describe())



=== 컬럼 정보 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    4803 non-null   int64         
 1   title                 4803 non-null   object        
 2   original_title        4803 non-null   object        
 3   release_date          4802 non-null   datetime64[ns]
 4   budget                4803 non-null   int64         
 5   revenue               4803 non-null   int64         
 6   profit                4803 non-null   int64         
 7   roi                   3766 non-null   float64       
 8   runtime               4801 non-null   float64       
 9   popularity            4803 non-null   float64       
 10  vote_average          4803 non-null   float64       
 11  vote_count            4803 non-null   int64         
 12  status                4803 non-null   object        
 13  gen

In [15]:
df.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [10]:
# 홈페이지와 태그라인의 결측치 처리방법

# overview, runtime, release_date 결측치 처리 방법
                        

In [11]:
def parse_json_column(df: pd.DataFrame, column: str, key: str = 'name') -> pd.DataFrame:
    """
    JSON 형식의 문자열이 들어있는 column을 파이썬 객체로 변환한 뒤,
    각 딕셔너리에서 지정한 key 값을 추출하여 리스트로 저장합니다.
    
    Parameters
    ----------
    df : pd.DataFrame
        대상 데이터프레임
    column : str
        JSON 문자열이 들어있는 컬럼명
    key : str, default 'name'
        딕셔너리에서 추출할 키(예: 'name')
    
    Returns
    -------
    pd.DataFrame
        원본 데이터프레임에 `{column}_list` 컬럼을 추가한 결과
    """
    # 1. JSON 문자열 → 파이썬 객체(리스트 of dict) 변환
    df[column + '_py'] = df[column].apply(ast.literal_eval)
    
    # 2. 각 dict에서 key값만 추출하여 리스트로 변환
    df[column + '_list'] = df[column + '_py'].apply(lambda lst: [d.get(key) for d in lst])
    
    # 3. (옵션) 분석 편의를 위해 공백 구분 문자열로 변환
    df[column + '_str'] = df[column + '_list'].apply(lambda names: ' '.join(names))
    
    # 4. 중간 컬럼 제거
    df.drop(columns=[column + '_py'], inplace=True)
    
    return df

# 사용 예시
if __name__ == '__main__':
    # CSV 파일 읽기
    movies_df = pd.read_csv('tmdb_5000_movies.csv')
    
    # genres 컬럼 정리
    movies_df = parse_json_column(movies_df, 'genres', key='name')
    
    # keywords 컬럼 정리
    movies_df = parse_json_column(movies_df, 'keywords', key='name')
    
    # production_companies 컬럼 정리
    movies_df = parse_json_column(movies_df, 'production_companies', key='name')
    
    # 결과 확인
    print(movies_df[['genres_list', 'genres_str']].head())

                                     genres_list  \
0  [Action, Adventure, Fantasy, Science Fiction]   
1                   [Adventure, Fantasy, Action]   
2                     [Action, Adventure, Crime]   
3               [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, Science Fiction]   

                                 genres_str  
0  Action Adventure Fantasy Science Fiction  
1                  Adventure Fantasy Action  
2                    Action Adventure Crime  
3               Action Crime Drama Thriller  
4          Action Adventure Science Fiction  


In [12]:
movies_df.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,tagline,title,vote_average,vote_count,genres_list,genres_str,keywords_list,keywords_str,production_companies_list,production_companies_str
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,Enter the World of Pandora.,Avatar,7.2,11800,"[Action, Adventure, Fantasy, Science Fiction]",Action Adventure Fantasy Science Fiction,"[culture clash, future, space war, space colon...",culture clash future space war space colony so...,"[Ingenious Film Partners, Twentieth Century Fo...",Ingenious Film Partners Twentieth Century Fox ...
